In [1]:
%%init_spark
# Configure Spark to use a local master
# launcher.master = "local[*]" # tested to work
launcher.master = "yarn"
launcher.packages = "neo4j-contrib:neo4j-spark-connector:2.1.0-M4"

In [2]:
println(sc.master)
// val rdd = sc.parallelize(0 to 999)
// rdd.takeSample(false, 5)

Intitializing Scala interpreter ...

Spark Web UI available at http://hadoop-master:8088/proxy/application_1551686720681_0005
SparkContext available as 'sc' (version = 2.0.0, master = yarn, app id = application_1551686720681_0005)
SparkSession available as 'spark'


yarn


In [3]:
import org.neo4j.spark._

import org.neo4j.spark._


In [4]:
val neo = Neo4j(sc)

neo: org.neo4j.spark.Neo4j = org.neo4j.spark.Neo4j@6ccf3165


In [5]:
val rdd = neo.cypher("MATCH (n:Person) return id(n) as id").loadRowRdd
rdd.count

rdd: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = Neo4jRDD partitions Partitions(1,9223372036854775807,9223372036854775807,None) MATCH (n:Person) return id(n) as id using Map()
res1: Long = 100


In [6]:
rdd.first.schema.fieldNames

res2: Array[String] = Array(id)


In [7]:
neo.cypher("MATCH (n:Person) RETURN id(n)").loadRdd[Long].mean

res3: Double = 66.5


In [8]:
neo.cypher("MATCH (n:Person) WHERE n.id <= {maxId} RETURN n.id").param("maxId", 10).loadRowRdd.count

res4: Long = 10


In [9]:
neo.nodes("MATCH (n:Person) RETURN id(n) SKIP {_skip} LIMIT {_limit}").partitions(4).batch(25).loadRowRdd.count

<console>: 32: error: overloaded method value nodes with alternatives:

In [10]:
neo.pattern("Person",Seq("KNOWS"),"Person").rows(80).batch(21).loadNodeRdds.count

res6: Long = 80


In [11]:
neo.pattern("Person",Seq("KNOWS"),"Person").partitions(12).batch(100).loadRelRdd.count

res7: Long = 1000


In [12]:
val df = neo.pattern("Person",Seq("KNOWS"),"Person").partitions(12).batch(100).loadDataFrame

df: org.apache.spark.sql.DataFrame = [id: bigint]


In [13]:
df.collect()

res8: Array[org.apache.spark.sql.Row] = Array([19], [29], [83], [55], [51], [104], [73], [44], [77], [17], [47], [34], [95], [40], [56], [89], [94], [27], [116], [84], [82], [80], [106], [49], [23], [30], [79], [109], [58], [90], [98], [81], [93], [86], [108], [96], [26], [22], [31], [71], [69], [60], [100], [105], [101], [62], [75], [88], [70], [91], [114], [37], [53], [38], [25], [92], [39], [45], [99], [107], [32], [35], [50], [36], [76], [43], [52], [42], [113], [111], [18], [21], [64], [103], [41], [57], [48], [24], [115], [66], [97], [110], [20], [28], [67], [59], [61], [33], [65], [78], [46], [87], [102], [68], [54], [74], [112], [72], [63], [85])


In [14]:
sc.stop()